In [1]:
from simanneal import Annealer
import random

In [2]:
class GroupingProblem(Annealer):

    def __init__(self, init_state, members, teams, skills, scores):
        super(GroupingProblem, self).__init__(init_state)  # important!
        self.members = members
        self.teams = teams
        self.scores = scores
        self.sum_scores = [sum(ai) for ai in scores]
        self.skills = skills

    # 探索点の移動ルール
    def move(self):
        # ランダムに選択したメンバーm1をチームt1からチームt2に移す
        m1 = random.choice(list(range(len(self.members))))
        t1 = self.state[m1].index(1)
        t2 = random.choice(list(range(len(self.teams))))
        self.state[m1][t1], self.state[m1][t2] = self.state[m1][t2], self.state[m1][t1]

    # 目的関数
    def energy(self):
        v = 0
        nu = sum(sum(self.sum_scores[i] * self.state[i][j] for i in range(len(self.members))) for j in range(len(self.teams))) / len(self.teams)
        for j in range(len(self.teams)):
            mu_j = sum(sum(self.scores[i][k] * self.state[i][j] for i in range(len(self.members))) for k in range(len(self.skills))) / len(self.skills)
            for k in range(len(self.skills)):
                v += abs(sum(self.scores[i][k] * self.state[i][j] for i in range(len(self.members))) - mu_j)
            v += 1.5 * abs(sum(self.sum_scores[i] * self.state[i][j] for i in range(len(self.members))) - nu)
        return v

In [ ]:

teams = ['A', 'B', 'C']
members = ['P', 'Q', 'R', 'S', 'T', 'U']
skills = ['Controller', 'Promoter', 'Supporter', 'Analyzer']
scores = [[6, 0, 1, 3],
            [2, -1, 3, 5],
            [2, 4, 0, 0],
            [3, 4, 5, 0],
            [0, 2, 1, 4],
            [2, 3, -1, 1]]

# 初期割当て
init_state = [[0 for j in range(len(teams))] for i in range(len(members))]
for i in range(len(members)):
    init_state[i][0] = 1  # 最初は全員Aチームに所属

prob = GroupingProblem(init_state, members, teams, skills, scores)
prob.steps = 100000
prob.copy_strategy = "deepcopy"
prob.anneal()  # 焼きなまし実行

for i,s in enumerate(prob.state):
    print(members[i], teams[s.index(1)])